# Model Builder Redesign
## This notebook highlights the new changes made to ModelBuilder and related utilities

- Latest Container Image Utility function
- Handshake with ModelTrainer 
- Unified Deployment from ModelBuilder

In [ ]:
alias = "user"

## Inital Setup

In [ ]:
!pip install sagemaker

In [ ]:
from sagemaker import Session, get_execution_role

sagemaker_session = Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
bucket = sagemaker_session.default_bucket()
default_bucket_prefix = sagemaker_session.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    default_bucket_prefix_path = f"{default_bucket_prefix}/"

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import os

import pandas as pd

# Prepare Data

iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df["target"] = iris.target

os.makedirs("./data", exist_ok=True)

iris_df = iris_df[["target"] + [col for col in iris_df.columns if col != "target"]]

train_data, test_data = train_test_split(iris_df, test_size=0.2, random_state=42)

train_data.to_csv("./data/train.csv", index=False, header=False)
test_data.to_csv("./data/test.csv", index=False, header=False)

# Remove the target column from the testing data. We will use this to call invoke_endpoint later
test_data_no_target = test_data.drop("target", axis=1)

prefix = "DEMO-scikit-iris"

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    prefix = f"{default_bucket_prefix}/{prefix}"

TRAIN_DATA = "train.csv"
TEST_DATA = "test.csv"
DATA_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    DATA_DIRECTORY, bucket=bucket, key_prefix="{}/{}".format(prefix, DATA_DIRECTORY)
)


s3_input_path = "s3://{}/{}/data/{}".format(bucket, prefix, TRAIN_DATA)
s3_output_path = "s3://{}/{}/output".format(bucket, prefix)

s3_test_path = "s3://{}/{}/data/{}".format(bucket, prefix, TEST_DATA)

print(s3_input_path)
print(s3_output_path)


# Integration with ModelTrainer

The handshake between ModelTrainer and ModelBuilder is made seamlessly as in this example. The created model trainer object is directly fed into the model attribute of ModelBuilder through resource chaining . Fetching of the model artifacts is done internally within the ModelBuilder. 

Note: 
- Other than the ModelTrainer, the ModelBuilder also supports chaining of attributes such as Estimator or sagemaker-core's TrainingJob into the model attribute. 

Other than this there is an upgrade designed for retrieving images for a particular framework. The enhanced `image_uris.retrieve()` method will fetch the latest version of an image automatically if the version is not provided.


In [ ]:
from sagemaker import image_uris
from sagemaker_core.main.shapes import (
    Channel,
    DataSource,
    S3DataSource,
    OutputDataConfig,
    StoppingCondition,
)
from sagemaker.modules.train.model_trainer import ModelTrainer

# xgboost_image="433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest"
xgboost_image = image_uris.retrieve(framework="xgboost", region="us-west-2", image_scope="training")
print(xgboost_image)

model_trainer = ModelTrainer(
    base_job_name=f"{alias}-mb-handshake",
    hyperparameters={
        "objective": "multi:softmax",
        "num_class": "3",
        "num_round": "10",
        "eval_metric": "merror",
    },
    training_image=xgboost_image,
    training_input_mode="File",
    role=role,
    output_data_config=OutputDataConfig(s3_output_path=s3_output_path),
    stopping_condition=StoppingCondition(max_runtime_in_seconds=600),
)

model_trainer.train(
    input_data_config=[
        Channel(
            channel_name="train",
            content_type="csv",
            compression_type="None",
            record_wrapper_type="None",
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type="S3Prefix",
                    s3_uri=s3_input_path,
                    s3_data_distribution_type="FullyReplicated",
                )
            ),
        )
    ],
)

In [ ]:
import numpy as np
from sagemaker.serve.builder.schema_builder import SchemaBuilder
import pandas as pd
from xgboost import XGBClassifier
from sagemaker.serve.spec.inference_spec import InferenceSpec
from sagemaker.serve import ModelBuilder

data = {"Name": ["Alice", "Bob", "Charlie"]}
df = pd.DataFrame(data)
schema_builder = SchemaBuilder(sample_input=df, sample_output=df)


class XGBoostSpec(InferenceSpec):
    def load(self, model_dir: str):
        print(model_dir)
        model = XGBClassifier()
        model.load_model(model_dir + "/xgboost-model")
        return model

    def invoke(self, input_object: object, model: object):
        prediction_probabilities = model.predict_proba(input_object)
        predictions = np.argmax(prediction_probabilities, axis=1)
        return predictions


model_builder = ModelBuilder(
    model=model_trainer,  # ModelTrainer object passed onto ModelBuilder directly
    role_arn=role,
    image_uri=xgboost_image,
    inference_spec=XGBoostSpec(),
    schema_builder=schema_builder,
    instance_type="ml.c6i.xlarge",
)
model = model_builder.build()

Once the model has been built , it can be deployed directly through the model_builder.deploy() method. This abstracts out information that was previously used commonly in workflows for different deployment modes. The deploy() method takes in an optional parameter `inference_config`. This determines attributes for modes such as serverless, async, batch and multi-model/multi-container endpoints. If the `inference_config` is not provided, the default real-time deployment is carried out.

## ModelBuilder - Real-Time Deployment

In [ ]:
predictor = model_builder.deploy(endpoint_name=f"{alias}-xgboost-deploy-realtime")
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0])
result = predictor.predict(sklearn_input)
print(result)

## ModelBuilder - Serverless Deployment


In [ ]:
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig

predictor = model_builder.deploy(
    endpoint_name=f"{alias}-xgboost-deploy-serverless",
    inference_config=ServerlessInferenceConfig(memory_size_in_mb=2048),
)
sklearn_input = np.array([1.0, 2.0, 3.0, 4.0])
result = predictor.predict(sklearn_input)
print(result)

## ModelBuilder - Async Deployment


In [ ]:
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.s3_utils import s3_path_join

predictor = model_builder.deploy(
    endpoint_name=f"{alias}-xgboost-deploy-async",
    inference_config=AsyncInferenceConfig(
        output_path=s3_path_join(
            "s3://", bucket, f"{default_bucket_prefix_path}async_inference/output"
        )
    ),
)

sklearn_input = np.array([1.0, 2.0, 3.0, 4.0])
result = predictor.predict(sklearn_input)
print(result)

## ModelBuilder - Batch Deployment


In [ ]:
from sagemaker.batch_inference.batch_transform_inference_config import BatchTransformInferenceConfig
from sagemaker.s3_utils import s3_path_join

transformer = model_builder.deploy(
    endpoint_name=f"{alias}-xgboost-deploy-batch",
    inference_config=BatchTransformInferenceConfig(
        instance_count=1,
        instance_type="ml.m5.large",
        output_path=s3_path_join(
            "s3://", bucket, f"{default_bucket_prefix_path}batch_inference/output"
        ),
        test_data_s3_path=s3_test_path,
    ),
)

print(transformer)

## ModelBuilder - Multi-Model Endpoint Deployment


In [ ]:
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

predictor = model_builder.deploy(
    endpoint_name=f"{alias}-xgboost-deploy-multi-model",
    inference_config=ResourceRequirements(
        requests={
            "num_cpus": 0.5,
            "memory": 512,
            "copies": 2,
        },
        limits={},
    ),
)

sklearn_input = np.array([1.0, 2.0, 3.0, 4.0])
result = predictor.predict(sklearn_input)
print(result)